<a href="https://colab.research.google.com/github/MaseedIrfanAli/DataFlow/blob/main/ParDo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 18.1 MB/s eta 0:00:00
 

In [19]:
import apache_beam as beam
p1 = beam.Pipeline()
customers = (
    p1
    |beam.io.ReadFromText('/content/Customer.txt')
    |beam.Map(lambda record:record.split(','))
    |beam.Filter(lambda record: record[2]=='New York' and int(record[3])>20)
    |beam.io.WriteToText('/content/result','asa',3)
)
p1.run()

In [20]:
!cat /content/result-00000-of-00001

['1', ' John Doe', 'New York', ' 30']
['4', ' Emily Davis', 'New York', ' 35']


In [22]:
import apache_beam as beam
p1 = beam.Pipeline()
def SplitRow(elem):
  return elem.split(',')
def FilterRow(record):
  return record[2]=='New York' and int(record[3])>20

customers = (
    p1
    |beam.io.ReadFromText('/content/Customer.txt')
    |beam.Map(SplitRow)
    |beam.Filter(FilterRow)
    |beam.io.WriteToText('/content/result','ram')
)
p1.run()

In [23]:
import apache_beam as beam
p1 = beam.Pipeline()


class SplitRow(beam.DoFn):
  def process(self,elem):
    return [elem.split(',')]

class FilterRow(beam.DoFn):
  def process(self,record):
    if record[2]=='New York' and int(record[3])>20
      return [record]
customers = (
    p1
    |beam.io.ReadFromText('/content/Customer.txt')
    |beam.ParDo(SplitRow())
    |beam.ParDo(FilterRow())
    |beam.io.WriteToText('/content/result','ram')
)
p1.run()

In [24]:
import apache_beam as beam
words=['peter','piper','pickled','picked','peck','pepper']
p1 = beam.Pipeline()
def FindWord(element):
  if element in words:
    return True
Freq = (
    p1
    |beam.io.ReadFromText('/content/Peter_Piper.txt')
    |beam.FlatMap(lambda record:record.split(' '))
    |beam.Filter(FindWord)
    |beam.Map(lambda record: (record,1))
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('/content/Peterresult')
)
p1.run()


In [25]:
import apache_beam as beam
words=['peter','piper','pickled','picked','peck','pepper']
p1 = beam.Pipeline()

def SplitRow(element):
  return element.split(' ')

def FindWord(element):
  if element in words:
    return True
Freq = (
    p1
    |beam.io.ReadFromText('/content/Peter_Piper.txt')
    |beam.FlatMap(SplitRow)
    |beam.Filter(FindWord)
    |beam.Map(lambda record: (record,1))
    |beam.CombinePerKey(sum)
    |beam.io.WriteToText('/content/Def_result')
)
p1.run()


In [4]:
import apache_beam as beam
words=['peter','piper','pickled','picked','peck','pepper']
p1 = beam.Pipeline()

def FindWord(element):
    if element in words:
      return True
class SplitRow(beam.DoFn):
  def process(self,element):
    return element.split(' ')

class CalculateFreq(beam.DoFn):
  def process(self,element):
    (key,values) = element
    return [(key,sum(values))]

Freq = (
    p1
    |'Read you input ' >> beam.io.ReadFromText('/content/Peter_Piper.txt')
    |'Split records with space' >> beam.ParDo(SplitRow())
    |'Filter records which match the words' >> beam.Filter(FindWord)
    |beam.Map(lambda record: (record,1))
    |beam.GroupByKey()
    |beam.ParDo(CalculateFreq())
    |beam.io.WriteToText('/content/Cal_result')
)
p1.run()


In [5]:
!cat /content/Cal_result-00000-of-00001


('picked', 2)
('peck', 4)
('pickled', 4)


In [31]:
import apache_beam
p1 = beam.Pipeline()

side_list = list()
with open('/content/customer_exclude.txt','r') as exclude_file:
  for cust_id in exclude_file:
    side_list.append(cust_id.rstrip())

print(side_list)

class SplitRow(beam.DoFn):
  def process(self,element,side_list):
   customer = element.split(',')
   if customer[0] not in side_list:
      return [customer]

customers = (
    p1
    |beam.io.ReadFromText('/content/customer.txt')
    |beam.ParDo(SplitRow(),side_list)
    |beam.io.WriteToText('/content/result','adfsad')
)
p1.run()

['1,John Smith,NY,28', '2,Emily Davis,CA,34', '3,Michael Brown,TX,21', '4,Linda Johnson,FL,45', '5,James Wilson,NY,19', '6,Patricia Garcia,CA,27', '7,Robert Martinez,TX,50', '8,Mary Anderson,FL,22', '9,William Thomas,NY,32', '10,Elizabeth Taylor,CA,40']


In [25]:
!cat /content/result-00000-of-00001

['3']
['7']
['10']


In [38]:
import apache_beam as beam
p1 = beam.Pipeline()

side_list = list()
with open ('/content/customer_exclude.txt','r') as exclude_file:
  for cust_id in exclude_file:
    side_list.append(cust_id.rstrip())

print(side_list)

class SplitRow(beam.DoFn):
  def process(self,element,side_list):
    customer = element.split(',')
    if customer[0] not in side_list:
      return [customer]


customers = (
    p1
    |beam.io.ReadFromText('/content/customer.txt')
    |beam.ParDo(SplitRow(),side_list)
    |beam.io.WriteToText('/content/result','ashjgffy')
)
p1.run()

['1,Johnsmith,NY,28', '2,EmilyDavis,CA,34', '3,MichaelBrown,TX,21', '4,LindaJohnson,FL,45', '5,JamesWilson,NY,19', '6,PatriciaGarcia,CA,27', '7,RobertMartinez,TX,50', '8,MaryAnderson,FL,22', '9,WilliamThomas,NY,32', '10,ElizabethTaylor,CA,40']


TypeError: AsSideInput.__init__() missing 1 required positional argument: 'pcoll'

In [89]:
import apache_beam as beam

# Load the exclusion list using `with open`
p1 = beam.Pipeline()
side_list = list()
with open('/content/customer_exclude.txt', 'r') as exclude_file:
    for cust_id in exclude_file:
        side_list.append(cust_id.rstrip())

print("Exclusion List:", side_list)

# Define a DoFn to process each element
class SplitRow(beam.DoFn):
     def process(self, element, side_list):
        custome = element.split(',')
        if custome[0] not in side_list:
            return [custome]

print(customers)  # Use 'yield' to produce output

# Define the pipeline


# Apply transformations to the pipeline
customer = (
    p1
    | 'Read Customers' >> beam.io.ReadFromText('/content/customer.txt')
    | 'Filter Customers' >> beam.ParDo(SplitRow(),side_list)
    | 'Write Results' >> beam.io.WriteToText('/content/result')

)

# Run the pipeline
p1.run().wait_until_finish()

Exclusion List: ['id, name, city, age', '1, John Doe, New York, 30', '2, Jane Smith, Los Angeles, 25', '3, Michael Johnson, Chicago, 40', '4, Emily Davis, Houston, 35', '5, David Brown, Phoenix, 28', '6, Jane Smith, Los Angeles, 25', '7, Michael Johnson, Chicago, 40', '8, Emily Davis, Houston, 35', '9, David Brown, Phoenix, 28', '10, David Brown, Phoenix, 28']
PCollection[[51]: Write Results/Write/WriteImpl/FinalizeWrite.None]


'DONE'

In [90]:
!cat /content/result-00000-of-00001

['3']
['7']
['10']
